# Base_Model
1. 直接使用圖片不做Face detector
2. 不做影像預處理

In [1]:
# 將警告訊息關掉
import warnings
warnings.filterwarnings('ignore')

#Utilities相關函式庫
import os
import sys

# 圖像處理函示庫
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

# 數值處理函式庫
import pandas as pd
import numpy as np

# 深度學習函式庫
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop, Adam, SGD

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

# 機器學習函式庫
from sklearn.model_selection import train_test_split


# 人臉偵測函式庫
from mtcnn import MTCNN

# 相關參數設定
picture_size = 224
batch_size = 32  # 一次用多少筆資料更新模型
num_classes = 5  # 資料集有幾個類別
epochs = 100      # 訓練迭代次數
save_dir = os.path.join(os.getcwd(), 'saved_models') # 儲存模型的路徑
model_name = 'Face_BaseModel, trained_model.h5' # 模型名稱

Using TensorFlow backend.


In [2]:
# 專案的根目錄路徑
ROOT_DIR = os.getcwd()

# 資料集目錄
DATA_PATH = os.path.join(ROOT_DIR, 'Data')

# 訓練/驗證的圖像資料目錄
TRAIN_IMGS_PATH = os.path.join(DATA_PATH, 'training_set')
TEST_IMGS_PATH = os.path.join(DATA_PATH, 'testing_set')

In [3]:
# 載入分類的 Label
# 用來對訓練資料即進行分類
dict_Category = {}
classmap = pd.read_csv('./Data/classmap.csv', header = None)

for idx, label in classmap.iterrows():
    dict_Category[label[0]] = label[1]
dict_Category

{'classname': 'classnum',
 'rika': '0',
 'risa': '1',
 'yui': '2',
 'akane': '3',
 'neru': '4'}

In [8]:
# 讀取資料 + Label
def load_data(path,label):
    imageList = []
    labelList = []
    imgs = os.listdir(path)
    num = len(imgs)
    
    for imgname in imgs:
        if(imgname[-4:] == ".png"):
            img = cv2.imread(path+ imgname, cv2.IMREAD_COLOR)                                                                                                                                                                                                                                        
            img = cv2.resize(img, (picture_size, picture_size))      
#             img = img.astype(np.float32)  # 可能導致圖片印不出來 詳細原因待釐清
            img = img[:,:,::-1] # 將OpenCV的BGR轉為RGB, 用灰階的話就不使用
            imageList.append(img)
            labelList.append(label)
            print(path+ imgname)
            
    return imageList,labelList

In [9]:
# 對訓練資料集進行類別 Label
TrainDirectory = TRAIN_IMGS_PATH
labelnames = os.listdir(TrainDirectory) # 列出有幾個要分類的人, 每個人的圖像放在一個資料集，因此五人則是五個資料集
total_img=[]
total_label=[]
print(labelnames)

for labelname in labelnames:
    imageList,labelList = load_data(TrainDirectory+'/'+labelname+'/',dict_Category[labelname])
    total_img = total_img + imageList;
    total_label = total_label+labelList;
    print(labelname,'-'*3,len(imageList))

total_image=np.asarray(total_img)


labelnames = keras.utils.to_categorical(total_label,len(labelnames))    

['akane', 'neru', 'rika', 'risa', 'yui']
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/000.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/001.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/002.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/003.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/004.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/005.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/006.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/007.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/008.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/009.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/010.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/akane/011.png
C:\Users\fo

C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/017.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/018.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/019.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/020.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/021.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/022.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/023.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/024.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/025.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/026.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/027.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/neru/028.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set

C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/011.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/012.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/013.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/014.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/015.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/016.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/017.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/018.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/019.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/020.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/021.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/rika/022.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set

C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/054.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/055.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/056.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/057.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/058.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/059.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/060.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/061.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/062.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/063.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/064.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/risa/065.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set

C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/067.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/068.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/069.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/070.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/071.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/072.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/073.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/074.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/075.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/076.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/077.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/078.png
C:\Users\foresight\Desktop\AI\Face Recognition\Data\training_set/yui/079.png

In [11]:
# 切分訓練/測試資料
#train_x = total_img
#train_y = labelnames

train_x,test_x, train_y, test_y = train_test_split(total_image,
                                                  labelnames,
                                                test_size = 0.1,stratify= labelnames)#random_state = 123,
print('train:',train_x.shape,'test:',test_x.shape)

# 正規化資料
# 標準化0~255的值到0~1
x_train_normalize = train_x.astype('float32') / 255.0
x_test_normalize = test_x.astype('float32') / 255.0


train: (484, 224, 224, 3) test: (54, 224, 224, 3)


In [12]:
# 建立CNN模型
# CNN層
model = Sequential()
model.add(Conv2D(64, (3, 3), padding = 'same', input_shape = x_train_normalize.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))

# DNN層
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 222, 222, 128)     73856     
_________________________________________________________________
activation_2 (Activation)    (None, 222, 222, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 128)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 111, 111, 128)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 111, 111, 256)    

In [13]:
# 訓練模型
optimizer = Adam(lr=3e-6,decay=0.0001,beta_1=0.99,beta_2=0.999) #100e-6
model.compile(loss = 'categorical_crossentropy',
             optimizer = optimizer,
             metrics = ['accuracy'])


In [14]:
# Data Augmentation

datagen  = ImageDataGenerator(
            rotation_range = 30,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            horizontal_flip = True, 
            vertical_flip = False,
            shear_range=0.2,
            zoom_range=0.2,
            fill_mode='nearest')

In [ ]:
# Use ModelCheckpoint to save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(model_path, monitor = 'val_loss', save_best_only = True, verbose = 1)

earlystop = EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 1) # patience=10為連續10次模型loss沒再下降就停止

# Fit model
# aug_ratio = 2
# steps_per_epoch = int(aug_ratio * train_x.shape[0] / batch_size)
# validation_steps = int(aug_ratio * test_x.shape[0] / batch_size)

model_history = model.fit_generator(datagen.flow(x_train_normalize, train_y, batch_size = batch_size),
                                   epochs = epochs,
                                   validation_data = (x_test_normalize, test_y),
                                   #workers = 1, # 使用的CPU數量
                                   callbacks = [checkpoint, earlystop],
#                                    steps_per_epoch=steps_per_epoch,
#                                    validation_steps=validation_steps
                                   )

# loading save model
print('Loading trained model')
model = load_model(model_path)

# Score trained model
scores = model.evaluate(x_test_normalize, test_y, verbose = 1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/100
16/16 [==============================] - 148s 9s/step - loss: 1.6203 - accuracy: 0.2252 - val_loss: 1.6042 - val_accuracy: 0.2037

Epoch 00001: val_loss improved from inf to 1.60421, saving model to C:\Users\foresight\Desktop\AI\Face Recognition\saved_models\Face_BaseModel, trained_model.h5
Epoch 2/100
16/16 [==============================] - 149s 9s/step - loss: 1.6022 - accuracy: 0.2335 - val_loss: 1.5991 - val_accuracy: 0.2222

Epoch 00002: val_loss improved from 1.60421 to 1.59909, saving model to C:\Users\foresight\Desktop\AI\Face Recognition\saved_models\Face_BaseModel, trained_model.h5
Epoch 3/100
16/16 [==============================] - 151s 9s/step - loss: 1.6022 - accuracy: 0.2521 - val_loss: 1.5945 - val_accuracy: 0.3333

Epoch 00003: val_loss improved from 1.59909 to 1.59450, saving model to C:\Users\foresight\Desktop\AI\Face Recognition\saved_models\Face_BaseModel, trained_model.h5
Epoch 4/100
16/16 [==============================] - 145s 9s/step - loss: 1.5979

Epoch 29/100
16/16 [==============================] - 146s 9s/step - loss: 1.5180 - accuracy: 0.3264 - val_loss: 1.5238 - val_accuracy: 0.3704

Epoch 00029: val_loss did not improve from 1.52374
Epoch 30/100
16/16 [==============================] - 144s 9s/step - loss: 1.4928 - accuracy: 0.3450 - val_loss: 1.5224 - val_accuracy: 0.3704

Epoch 00030: val_loss improved from 1.52374 to 1.52236, saving model to C:\Users\foresight\Desktop\AI\Face Recognition\saved_models\Face_BaseModel, trained_model.h5
Epoch 31/100
16/16 [==============================] - 146s 9s/step - loss: 1.4919 - accuracy: 0.3037 - val_loss: 1.5217 - val_accuracy: 0.3519

Epoch 00031: val_loss improved from 1.52236 to 1.52173, saving model to C:\Users\foresight\Desktop\AI\Face Recognition\saved_models\Face_BaseModel, trained_model.h5
Epoch 32/100
16/16 [==============================] - 148s 9s/step - loss: 1.5043 - accuracy: 0.3574 - val_loss: 1.5206 - val_accuracy: 0.3519

Epoch 00032: val_loss improved from 1.52173

Epoch 60/100
16/16 [==============================] - 157s 10s/step - loss: 1.4885 - accuracy: 0.3988 - val_loss: 1.4779 - val_accuracy: 0.3333

Epoch 00060: val_loss improved from 1.48176 to 1.47786, saving model to C:\Users\foresight\Desktop\AI\Face Recognition\saved_models\Face_BaseModel, trained_model.h5
Epoch 61/100
16/16 [==============================] - 154s 10s/step - loss: 1.4315 - accuracy: 0.3802 - val_loss: 1.4768 - val_accuracy: 0.3519

Epoch 00061: val_loss improved from 1.47786 to 1.47682, saving model to C:\Users\foresight\Desktop\AI\Face Recognition\saved_models\Face_BaseModel, trained_model.h5
Epoch 62/100
16/16 [==============================] - 153s 10s/step - loss: 1.4240 - accuracy: 0.3822 - val_loss: 1.4791 - val_accuracy: 0.3519

Epoch 00062: val_loss did not improve from 1.47682
Epoch 63/100
16/16 [==============================] - 153s 10s/step - loss: 1.4378 - accuracy: 0.3822 - val_loss: 1.4792 - val_accuracy: 0.3519

Epoch 00063: val_loss did not improve f